In [1]:
import os
import pandas as pd

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()

In [2]:
# read in segments layer
sdfSegmentsWithFactors = pd.DataFrame.spatial.from_featureclass('../results/layers/Master_Segs_withFactors_20211119.shp')

#create map1 that centers on Salt Lake
map1 = gis.map('Salt Lake')

# add _df to map
sdfSegmentsWithFactors.spatial.plot(map_widget = map1)

#show map1
map1

MapView(layout=Layout(height='400px', width='100%'))

In [3]:
# delete CO_FIPS=0 ... mostly for transit segs that have since been corrected
sdfSegmentsWithFactors = sdfSegmentsWithFactors[sdfSegmentsWithFactors['CO_FIPS']>0].copy()

# create only two classes, freeway or arterial
sdfSegmentsWithFactors.loc[sdfSegmentsWithFactors['FC_GROUP'].str.contains('Freeway', na=False), 'FC_GROUP'] = 'Freeway'
sdfSegmentsWithFactors.loc[~sdfSegmentsWithFactors['FC_GROUP'].str.contains('Freeway', na=False), 'FC_GROUP'] = 'Arterial'

sdfSegmentsWithFactors

,FID,Id,SEGID,BMP,EMP,DISTANCE,FULLNAME,FNCLASS,FC_CODE,FC_GROUP,...,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL,FAC_MAXMO,FAC_MAX,FACMANADJ,SUTRUCKS,CUTRUCKS,SHAPE
0,0,0,0006_000.0,0.000,0.665,0.666641,HWY 6,Arterial,3,Arterial,...,0.8495,1.0276,1.0914,1.0316,5,1.1065,0,0.2496,0.2324,"{""paths"": [[[236177.7000000002, 4327541.25], [..."
1,1,0,0006_000.7,0.665,16.022,15.369839,HWY 6,Arterial,3,Arterial,...,0.8495,1.0276,1.0914,1.0316,5,1.1065,0,0.2496,0.2324,"{""paths"": [[[237241.1799999997, 4327399.720000..."
2,2,0,0006_016.0,16.022,46.017,30.001961,HWY 6,Arterial,3,Arterial,...,0.8495,1.0276,1.0914,1.0316,5,1.1065,0,0.2496,0.2324,"{""paths"": [[[261403.90000000037, 4327045.4], [..."
3,3,0,0006_046.0,46.017,60.218,14.194306,HWY 6,Arterial,3,Arterial,...,0.8495,1.0276,1.0914,1.0316,5,1.1065,0,0.1751,0.3338,"{""paths"": [[[305380.16000000015, 4325741.85], ..."
4,4,0,0006_060.2,60.218,77.545,17.323237,HWY 6,Arterial,3,Arterial,...,0.8495,1.0276,1.0914,1.0316,5,1.1065,0,0.1751,0.3338,"{""paths"": [[[324618, 4337936.9], [324647.40000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8558,8558,0,WFRC_8219,0.000,0.000,1.154153,,Arterial,50,Arterial,...,0.8994,1.0107,1.0575,1.0324,8,1.0724,0,0.1278,0.0879,"{""paths"": [[[419504.53110000025, 4554272.0693]..."
8559,8559,0,WFRC_8220,0.000,0.000,0.192948,AIRPORT RD,Arterial,4,Arterial,...,0.9370,1.0104,1.0283,1.0243,9,1.0434,0,0.1143,0.0577,"{""paths"": [[[415710.1200000001, 4495585.710000..."
8560,8560,0,WFRC_8221,0.000,0.000,0.442365,9400 S,Arterial,2,Arterial,...,0.9653,1.0117,1.0135,1.0095,5,1.0245,0,0.1097,0.0482,"{""paths"": [[[423366.5, 4492605.300000001], [42..."
8561,8561,0,WFRC_8222,0.000,0.000,0.442365,300 W,Arterial,5,Arterial,...,0.9653,1.0117,1.0135,1.0095,5,1.0245,0,0.1097,0.0482,"{""paths"": [[[423433.7000000002, 4493180.6], [4..."


In [4]:
# calculate 2019 VMT
sdfSegmentsWithFactors['VMT'] = sdfSegmentsWithFactors['DISTANCE'] * sdfSegmentsWithFactors['AADT2019']

# Saturday and Sunday

sdfSegmentsWithFactors['FAC_MAY_SUN_x_VMT'  ] = sdfSegmentsWithFactors['FAC_MAY'] * sdfSegmentsWithFactors['FAC_SUN'  ] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_JUN_SUN_x_VMT'  ] = sdfSegmentsWithFactors['FAC_JUN'] * sdfSegmentsWithFactors['FAC_SUN'  ] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_JUL_SUN_x_VMT'  ] = sdfSegmentsWithFactors['FAC_JUL'] * sdfSegmentsWithFactors['FAC_SUN'  ] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_AUG_SUN_x_VMT'  ] = sdfSegmentsWithFactors['FAC_AUG'] * sdfSegmentsWithFactors['FAC_SUN'  ] * sdfSegmentsWithFactors['VMT']

sdfSegmentsWithFactors['FAC_MAY_SAT_x_VMT'  ] = sdfSegmentsWithFactors['FAC_MAY'] * sdfSegmentsWithFactors['FAC_SAT'  ] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_JUN_SAT_x_VMT'  ] = sdfSegmentsWithFactors['FAC_JUN'] * sdfSegmentsWithFactors['FAC_SAT'  ] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_JUL_SAT_x_VMT'  ] = sdfSegmentsWithFactors['FAC_JUL'] * sdfSegmentsWithFactors['FAC_SAT'  ] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_AUG_SAT_x_VMT'  ] = sdfSegmentsWithFactors['FAC_AUG'] * sdfSegmentsWithFactors['FAC_SAT'  ] * sdfSegmentsWithFactors['VMT']

sdfSegmentsWithFactors['FAC_MAY_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_MAY'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_JUN_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_JUN'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_JUL_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_JUL'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']
sdfSegmentsWithFactors['FAC_AUG_WEAVG_x_VMT'] = sdfSegmentsWithFactors['FAC_AUG'] * sdfSegmentsWithFactors['FAC_WEAVG'] * sdfSegmentsWithFactors['VMT']

display(sdfSegmentsWithFactors)

,FID,Id,SEGID,BMP,EMP,DISTANCE,FULLNAME,FNCLASS,FC_CODE,FC_GROUP,...,FAC_JUL_SUN_x_VMT,FAC_AUG_SUN_x_VMT,FAC_MAY_SAT_x_VMT,FAC_JUN_SAT_x_VMT,FAC_JUL_SAT_x_VMT,FAC_AUG_SAT_x_VMT,FAC_MAY_WEAVG_x_VMT,FAC_JUN_WEAVG_x_VMT,FAC_JUL_WEAVG_x_VMT,FAC_AUG_WEAVG_x_VMT
0,0,0,0006_000.0,0.000,0.665,0.666641,HWY 6,Arterial,3,Arterial,...,263.403828,265.868054,325.986870,325.456571,318.061839,321.037408,297.976918,297.492183,290.732833,293.452731
1,1,0,0006_000.7,0.665,16.022,15.369839,HWY 6,Arterial,3,Arterial,...,6072.949036,6129.763398,7515.842376,7503.615971,7333.125557,7401.729269,6870.054436,6858.878568,6703.037297,6765.746334
2,2,0,0006_016.0,16.022,46.017,30.001961,HWY 6,Arterial,3,Arterial,...,11854.410280,11965.312044,14670.941348,14647.075380,14314.277704,14448.192307,13410.361826,13388.546506,13084.343992,13206.752175
3,3,0,0006_046.0,46.017,60.218,14.194306,HWY 6,Arterial,3,Arterial,...,5027.352371,5074.384845,6221.818718,6211.697368,6070.560766,6127.352785,5687.217899,5677.966211,5548.956569,5600.868815
4,4,0,0006_060.2,60.218,77.545,17.323237,HWY 6,Arterial,3,Arterial,...,6135.560047,6192.960155,7593.329357,7580.976901,7408.728761,7478.039765,6940.883460,6929.592370,6772.144404,6835.499960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8558,8558,0,WFRC_8219,0.000,0.000,1.154153,,Arterial,50,Arterial,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8559,8559,0,WFRC_8220,0.000,0.000,0.192948,AIRPORT RD,Arterial,4,Arterial,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8560,8560,0,WFRC_8221,0.000,0.000,0.442365,9400 S,Arterial,2,Arterial,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8561,8561,0,WFRC_8222,0.000,0.000,0.442365,300 W,Arterial,5,Arterial,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [5]:
# aggregate to county, month, 

sdfCoAgg = sdfSegmentsWithFactors.groupby(['CO_FIPS','FC_GROUP'], as_index=False).agg(
    SUM_FAC_MAY_SAT_x_VMT  =('FAC_MAY_SAT_x_VMT'  , 'sum'),
    SUM_FAC_JUN_SAT_x_VMT  =('FAC_JUN_SAT_x_VMT'  , 'sum'),
    SUM_FAC_JUL_SAT_x_VMT  =('FAC_JUL_SAT_x_VMT'  , 'sum'),
    SUM_FAC_AUG_SAT_x_VMT  =('FAC_AUG_SAT_x_VMT'  , 'sum'),
    SUM_FAC_MAY_SUN_x_VMT  =('FAC_MAY_SUN_x_VMT'  , 'sum'),
    SUM_FAC_JUN_SUN_x_VMT  =('FAC_JUN_SUN_x_VMT'  , 'sum'),
    SUM_FAC_JUL_SUN_x_VMT  =('FAC_JUL_SUN_x_VMT'  , 'sum'),
    SUM_FAC_AUG_SUN_x_VMT  =('FAC_AUG_SUN_x_VMT'  , 'sum'),
    SUM_FAC_MAY_WEAVG_x_VMT=('FAC_MAY_WEAVG_x_VMT', 'sum'),
    SUM_FAC_JUN_WEAVG_x_VMT=('FAC_JUN_WEAVG_x_VMT', 'sum'),
    SUM_FAC_JUL_WEAVG_x_VMT=('FAC_JUL_WEAVG_x_VMT', 'sum'),
    SUM_FAC_AUG_WEAVG_x_VMT=('FAC_AUG_WEAVG_x_VMT', 'sum'),
    SUM_VMT                =('VMT'                , 'sum')
)

sdfCoAgg

,CO_FIPS,FC_GROUP,SUM_FAC_MAY_SAT_x_VMT,SUM_FAC_JUN_SAT_x_VMT,SUM_FAC_JUL_SAT_x_VMT,SUM_FAC_AUG_SAT_x_VMT,SUM_FAC_MAY_SUN_x_VMT,SUM_FAC_JUN_SUN_x_VMT,SUM_FAC_JUL_SUN_x_VMT,SUM_FAC_AUG_SUN_x_VMT,SUM_FAC_MAY_WEAVG_x_VMT,SUM_FAC_JUN_WEAVG_x_VMT,SUM_FAC_JUL_WEAVG_x_VMT,SUM_FAC_AUG_WEAVG_x_VMT,SUM_VMT
0,1,Arterial,1.894197e+05,1.906162e+05,1.867146e+05,1.880036e+05,1.553224e+05,1.564222e+05,1.532895e+05,1.541393e+05,1.723691e+05,1.735172e+05,1.700001e+05,1.710695e+05,1.672498e+05
1,1,Freeway,6.931700e+05,7.647695e+05,7.941837e+05,7.304643e+05,8.268797e+05,9.122904e+05,9.473785e+05,8.713678e+05,7.600591e+05,8.385677e+05,8.708203e+05,8.009521e+05,6.702635e+05
2,3,Arterial,1.064390e+06,1.125053e+06,1.130839e+06,1.121637e+06,8.033236e+05,8.512316e+05,8.561544e+05,8.467876e+05,9.338376e+05,9.881212e+05,9.934759e+05,9.841920e+05,9.990211e+05
3,3,Freeway,1.910982e+06,2.143297e+06,2.289204e+06,2.175399e+06,1.843710e+06,2.069367e+06,2.210993e+06,2.100512e+06,1.877355e+06,2.106342e+06,2.250109e+06,2.137965e+06,1.766754e+06
4,5,Arterial,1.982558e+06,2.246740e+06,2.476181e+06,2.367981e+06,1.409081e+06,1.596049e+06,1.756404e+06,1.679048e+06,1.695799e+06,1.921367e+06,2.116257e+06,2.023484e+06,1.935014e+06
5,7,Arterial,6.964346e+05,7.514908e+05,7.583448e+05,7.393960e+05,5.558451e+05,6.020115e+05,6.084338e+05,5.916072e+05,6.261150e+05,6.767239e+05,6.833616e+05,6.654748e+05,6.411137e+05
6,7,Freeway,7.350086e+04,7.417780e+04,7.230946e+04,7.473481e+04,5.293630e+04,5.342609e+04,5.207948e+04,5.383094e+04,6.321518e+04,6.379851e+04,6.219112e+04,6.427941e+04,7.934665e+04
7,9,Arterial,9.271645e+04,1.267540e+05,1.350812e+05,1.188176e+05,8.030048e+04,1.101408e+05,1.174629e+05,1.031966e+05,8.650527e+04,1.184427e+05,1.262669e+05,1.110027e+05,7.524209e+04
8,11,Arterial,2.772602e+06,2.802988e+06,2.739330e+06,2.808449e+06,1.890394e+06,1.914321e+06,1.871944e+06,1.917207e+06,2.331481e+06,2.358637e+06,2.305619e+06,2.362810e+06,2.955733e+06
9,11,Freeway,4.256994e+06,4.362699e+06,4.322877e+06,4.355973e+06,3.000294e+06,3.075589e+06,3.047822e+06,3.071060e+06,3.628625e+06,3.719124e+06,3.685330e+06,3.713496e+06,4.449560e+06


In [6]:
sdfCoAgg['FAC_MAY_SAT'  ] = sdfCoAgg['SUM_FAC_MAY_SAT_x_VMT'  ] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_JUN_SAT'  ] = sdfCoAgg['SUM_FAC_JUN_SAT_x_VMT'  ] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_JUL_SAT'  ] = sdfCoAgg['SUM_FAC_JUL_SAT_x_VMT'  ] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_AUG_SAT'  ] = sdfCoAgg['SUM_FAC_AUG_SAT_x_VMT'  ] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_MAY_SUN'  ] = sdfCoAgg['SUM_FAC_MAY_SUN_x_VMT'  ] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_JUN_SUN'  ] = sdfCoAgg['SUM_FAC_JUN_SUN_x_VMT'  ] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_JUL_SUN'  ] = sdfCoAgg['SUM_FAC_JUL_SUN_x_VMT'  ] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_AUG_SUN'  ] = sdfCoAgg['SUM_FAC_AUG_SUN_x_VMT'  ] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_MAY_WEAVG'] = sdfCoAgg['SUM_FAC_MAY_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_JUN_WEAVG'] = sdfCoAgg['SUM_FAC_JUN_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_JUL_WEAVG'] = sdfCoAgg['SUM_FAC_JUL_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']
sdfCoAgg['FAC_AUG_WEAVG'] = sdfCoAgg['SUM_FAC_AUG_WEAVG_x_VMT'] / sdfCoAgg['SUM_VMT']

dfCoAgg = sdfCoAgg[['CO_FIPS'      ,
                    'FC_GROUP'     ,
                    'FAC_MAY_SAT'  ,
                    'FAC_JUN_SAT'  ,
                    'FAC_JUL_SAT'  ,
                    'FAC_AUG_SAT'  ,
                    'FAC_MAY_SUN'  ,
                    'FAC_JUN_SUN'  ,
                    'FAC_JUL_SUN'  ,
                    'FAC_AUG_SUN'  ,
                    'FAC_MAY_WEAVG',
                    'FAC_JUN_WEAVG',
                    'FAC_JUL_WEAVG',
                    'FAC_AUG_WEAVG']]

display(dfCoAgg)

,CO_FIPS,FC_GROUP,FAC_MAY_SAT,FAC_JUN_SAT,FAC_JUL_SAT,FAC_AUG_SAT,FAC_MAY_SUN,FAC_JUN_SUN,FAC_JUL_SUN,FAC_AUG_SUN,FAC_MAY_WEAVG,FAC_JUN_WEAVG,FAC_JUL_WEAVG,FAC_AUG_WEAVG
0,1,Arterial,1.132556,1.139709,1.116382,1.124088,0.928685,0.935261,0.916530,0.921611,1.030609,1.037473,1.016444,1.022838
1,1,Freeway,1.034175,1.140998,1.184883,1.089817,1.233664,1.361092,1.413442,1.300038,1.133971,1.251102,1.299221,1.194981
2,3,Arterial,1.065433,1.126155,1.131947,1.122736,0.804111,0.852066,0.856993,0.847617,0.934753,0.989089,0.994449,0.985156
3,3,Freeway,1.081635,1.213127,1.295712,1.231297,1.043558,1.171282,1.251444,1.188910,1.062602,1.192210,1.273584,1.210110
4,5,Arterial,1.024570,1.161097,1.279671,1.223754,0.728202,0.824826,0.907696,0.867719,0.876376,0.992947,1.093665,1.045720
5,7,Arterial,1.086289,1.172165,1.182855,1.153299,0.866999,0.939009,0.949026,0.922780,0.976605,1.055544,1.065898,1.037998
6,7,Freeway,0.926326,0.934857,0.911311,0.941877,0.667152,0.673325,0.656354,0.678427,0.796696,0.804048,0.783790,0.810109
7,9,Arterial,1.232242,1.684616,1.795287,1.579137,1.067228,1.463818,1.561132,1.371528,1.149693,1.574155,1.678142,1.475275
8,11,Arterial,0.938042,0.948322,0.926785,0.950170,0.639568,0.647664,0.633326,0.648640,0.788799,0.797987,0.780050,0.799399
9,11,Freeway,0.956722,0.980479,0.971529,0.978967,0.674290,0.691212,0.684971,0.690194,0.815502,0.835841,0.828246,0.834576


In [7]:
dfCoAgg.to_csv('Sat-Sun-Factors-ByCounty-ByMonth.csv', index=False)